# Object Classification

### Problem Statement

In this tutorial, we shall train a model for classification on dataset using our own network, and then we shall evaluate how good/bad it is compared to the the one trained using transfer learning.

We use the famous CIFAR10 dataset readily avaliable from one of the frameworks. 

We shall follow the following steps: 
1. Load the Data (already split into train and test)
2. Define a Convolutional Neural Network to train a Classification Problem for classifying into 10 classes from the image input
3. Train a model using the training set, and created validation set
4. Evluate the model on test set
5. Repeat a similar approach for Tranfer Learning using a pre-trained model

In [ ]:
import tensorflow as tf
import keras
from keras.datasets import cifar10
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
tf.logging.set_verbosity(tf.logging.ERROR)

## Load the Data

In [ ]:
# Load Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Mention the Class Name List
class_list = ['airplane','automobile','bird','cat','deer','dog',
              'frog','horse','ship','truck']

# # Convert a one-hot vector for the test-labels
y_train = keras.utils.to_categorical(y_train, num_classes=len(class_list))


# # Split the test set to Validation & Test set
num_test_samples = x_test.shape[0]
x_val, y_val = x_test[0:num_test_samples//2,:,:], y_test[0:num_test_samples//2]
x_test, y_test = x_test[num_test_samples//2:,:,:], y_test[num_test_samples//2:]

y_val = keras.utils.to_categorical(y_val, num_classes=len(class_list))

## Visualize Data

In [ ]:
# Train Data 
x_train.shape, y_train.shape

In [ ]:
# Validation Data
x_val.shape, y_val.shape

In [ ]:
num = random.randint(0,60000)
class_fig = class_list[np.argmax(y_train[num])]

plt.imshow(x_train[num].reshape(32,32))
print("Class Name :{}".format(class_fig))

# Define the Model

In [ ]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers import Input, Conv2D, Activation, Dense, Flatten, MaxPooling2D

# Writing the model
model = Sequential(name='CIFAR-Classifier')

inputs = Input(shape=(32, 32, 3))

model.add(Conv2D(filters=32,kernel_size=3, strides=2,padding="same"))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))

model.add(Conv2D(filters=64,kernel_size=(3,3), strides=(2,2),padding="same"))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))

model.add(Conv2D(filters=128,kernel_size=(3,3), strides=(2,2),padding="same"))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(10,activation='softmax',use_bias=True))

# Compile the Model with the Loss function, Optimizer and Accuracy Metric
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# Check the Architecture
model.build(input_shape=(None, 32, 32, 3))
model.summary()

## Train the Model

In [ ]:
# Train the model
model.fit(x_train, y_train, validation_data=(x_val,y_val), epochs=7)

## Test the Model

In [ ]:
# Predict on Test Set
y_pred = model.predict_classes(x_test)
acc = np.sum(y_test == y_pred) / y_test.shape[0]
# Print Accuracy
print("Test Accuracy :{}".format(acc*100))